In [1]:
# install requirements
!pip install diffusers einops gradio typing_extensions  fastapi==0.100.0

# install ip-adapter
%cd /content
!git clone https://github.com/sagiodev/IP-Adapter-Negative.git

# download the models
%cd IP-Adapter-Negative
%mkdir models
%cd models
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin
%mkdir image_encoder
%cd image_encoder
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/pytorch_model.bin
!wget -nc https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/config.json
%cd /content/IP-Adapter-Negative

/content
fatal: destination path 'IP-Adapter-Negative' already exists and is not an empty directory.
/content/IP-Adapter-Negative
mkdir: cannot create directory ‘models’: File exists
/content/IP-Adapter-Negative/models
File ‘ip-adapter_sd15.bin’ already there; not retrieving.

File ‘ip-adapter-plus_sd15.bin’ already there; not retrieving.

mkdir: cannot create directory ‘image_encoder’: File exists
/content/IP-Adapter-Negative/models/image_encoder
File ‘pytorch_model.bin’ already there; not retrieving.

File ‘config.json’ already there; not retrieving.

/content/IP-Adapter-Negative


In [2]:
# Run Negative IP adapter demo
share_public = True # whether creates a public link

import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipelineLegacy, DDIMScheduler, AutoencoderKL
from PIL import Image

from ip_adapter import IPAdapterPlus, IPAdapter
base_model_path = "SG161222/Realistic_Vision_V4.0_noVAE"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "models/image_encoder"
ip_ckpt = "models/ip-adapter-plus_sd15.bin"
device = "cuda"
noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)

import random
ip_adapters_list = ['IP Adapter', 'IP Adapter Plus']
def generate(ip_adapter, image,
             neg_image,
             scale,
             scale_start,
             scale_stop,
             neg_scale,
             scale_neg_start,
             scale_neg_stop,
             prompt,
             negative_prompt,
             num_samples,
             steps,
             seed):
    if image is None and neg_image is None:
      scale =0
      neg_scale = 0
      image = Image.open("assets/images/statue.png")

    if seed == -1:
      seed = random.randint(0, 1e12)

    if ip_adapter == 'IP Adapter':
      # load ip-adapter
      ip_ckpt = "models/ip-adapter_sd15.bin"
      ip_model = IPAdapter(pipe, image_encoder_path, ip_ckpt, device)
    elif ip_adapter == 'IP Adapter Plus':
      # load ip-adapter
      ip_ckpt = "models/ip-adapter-plus_sd15.bin"
      ip_model = IPAdapterPlus(pipe, image_encoder_path, ip_ckpt, device, num_tokens=16)
    else:
      raise ValueError('IP adapter %s is undefined.'%ip_adapter)
    import gc
    gc.collect()

    images = ip_model.generate(pil_image=image,
                           negative_pil_image=neg_image,
                           prompt = prompt,
                           negative_prompt= negative_prompt,
                           num_samples=num_samples,
                           scale=scale,  # weight for image prompt
                           scale_start= scale_start,
                           scale_stop= scale_stop,
                           scale_neg = neg_scale, # weight for negative image prompt
                           scale_neg_start = scale_neg_start,
                           scale_neg_stop = scale_neg_stop,
                           num_inference_steps=steps,
                           seed=seed)
    print(images)
    return images


shortcut_js = """
<script>
function shortcuts(e) {

    if ((event.keyCode == 10 || event.keyCode == 13) && event.ctrlKey) {
        document.getElementById("generate-button").click();
    }
}
document.addEventListener('keyup', shortcuts, false);
</script>
"""

import gradio as gr
with gr.Blocks(head=shortcut_js) as demo:
  resized_image_file = gr.File(visible=False)

  gr.Markdown("# Stable Diffuson Negative Image Prompt")
  with gr.Row():
    with gr.Column():
      with gr.Row():
        with gr.Column():
          prompt = gr.Textbox(label="Prompt", value = "")
          scale = gr.Slider(0, 2, value=1, label="Image prompt weight")
          scale_start = gr.Slider(0, 1, value=0, label="Starting step")
          scale_stop = gr.Slider(0, 1, value=1, label="Ending step")
          imagePrompt = gr.Image(label="Image Prompt", type = "pil")
        with gr.Column():
          negativePrompt = gr.Textbox(label="Negative Prompt", value = "")
          negativeScale = gr.Slider(0, 2, value=1, label="Negative Image prompt weight")
          negativeScale_start = gr.Slider(0, 1, value=0, label="Starting step")
          negativeScale_stop = gr.Slider(0, 1, value=1, label="Ending step")
          negativeImagePrompt = gr.Image(label="Negative Image Prompt", type = "pil")
      with gr.Accordion(label="Advanced options", open=False):
        ip_adapter = gr.Dropdown(choices = ip_adapters_list, value = ip_adapters_list[0], label = "IP Adapter", interactive = True)
        numImages = gr.Number(precision=0, label="Number of images", value=1)
        steps = gr.Number(precision=0, label="Number of steps", value=25)
        seed = gr.Number(precision=0, label="Seed", value=-1)
    with gr.Column():
      btn = gr.Button("Generate", elem_id="generate-button")
      imagesOut = gr.Gallery(object_fit="contain", height="auto", selected_index = 0, preview = True)


  inputs = [ip_adapter, imagePrompt, negativeImagePrompt, scale, scale_start, scale_stop, negativeScale, negativeScale_start, negativeScale_stop, prompt, negativePrompt, numImages, steps, seed]
  outputs = imagesOut
  btn.click(generate, inputs=inputs, outputs=outputs)
  demo.queue().launch(debug=True, share=share_public, inline=False, show_error=True)


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to inst

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8a003c34dc32769514.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8a003c34dc32769514.gradio.live
